## Analyzing VELMA simulated runoff output

This notebook examines the observed runoff at the Ellsworth Creek gauge (found [here](https://fortress.wa.gov/ecy/eap/flows/station.asp?sta=24M050&historical=true#block2)) and compares it to the runoff simulated in VELMA. The stream gauge data comes with quality tags that describe how far each daily measurement deviates from a rating table for the creek. To determine if these outlier days are causing the discrepancy between simulated and observed runoff, days with very high (>2x rating table) or very low (&lt;1/2x) flows are removed and then imputed with a model trained on non-outlier data.

In [47]:
%matplotlib widget

import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import tempfile
import datetime
from sklearn.svm import SVR
import geopandas as gpd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from matplotlib.font_manager import FontProperties
import seaborn as sns
# import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib

In [2]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 140

In [3]:
# Import and format observed data (2003-2007 runoff)

input_dir = config.velma_data
results_dir = config.data_path.parents[0] / 'results' / 'ellsworth_baseline_03_07_12'

runoff_path = input_dir / 'runoff' / 'ellsworth_Q_2003_2007_dummy.csv'
runoff_start = pd.to_datetime('01-01-2003')
runoff_end = pd.to_datetime('12-31-2007')
nse_start = pd.to_datetime('01-01-2004')
nse_end = pd.to_datetime('12-31-2007')

runoff_obs = pd.read_csv(runoff_path, names=['runoff_obs'])
runoff_obs.index = pd.date_range(runoff_start, runoff_end)
runoff_obs['doy'], runoff_obs['year'] = runoff_obs.index.dayofyear, runoff_obs.index.year
runoff_obs = runoff_obs[(runoff_obs.index >= nse_start) & (runoff_obs.index <= nse_end)]

flow_path = input_dir.parents[0] / 'hydrology' / 'ellsworth' / 'wa_ecy_gauge' / 'streamflow' / 'ells_streamflow_2003_2008.csv'
quality = pd.read_csv(flow_path, usecols=['Date', 'Quality'], parse_dates=True, index_col=0)
quality = quality[(quality.index >= nse_start) & (quality.index <= nse_end)]

precip_path = input_dir / 'precip' / 'PRISM_gauge_avg_ppt_2003_2019.csv'
forcing_start = pd.to_datetime('01-01-2003')
forcing_end = pd.to_datetime('12-31-2019')                     
precip = pd.read_csv(precip_path, names=['precip'])
precip.index = pd.date_range(forcing_start, forcing_end)
precip['doy'], precip['year'] = precip.index.dayofyear, precip.index.year
precip = precip[(precip.index >= nse_start) & (precip.index <= nse_end)]

temp_path = input_dir / 'temp' / 'ellsworth_temp_2003_2019.csv'
temp = pd.read_csv(temp_path, names=['temp'])
temp.index = pd.date_range(forcing_start, forcing_end)
temp['doy'], temp['year'] = temp.index.dayofyear, temp.index.year
temp = temp[(temp.index >= nse_start) & (temp.index <= nse_end)]

# Import VELMA outputs
velma_results = pd.read_csv(results_dir / 'DailyResults.csv')

# Format datetime of results
jday_pad = velma_results['Day'].apply(lambda x: str(x).zfill(3))
str_year = velma_results['Year'].apply(lambda x: str(x))
velma_results['year_jday'] = str_year + jday_pad
velma_results.index = pd.to_datetime(velma_results['year_jday'], format='%Y%j')
velma_results = velma_results[(velma_results.index >= nse_start) & 
                              (velma_results.index <= nse_end)]

## Metrics

Calculate mean-squared error, root mean-squared error, and R-squared (equal to NSE) of simulated and observed runoff

In [48]:
# velma_results['Runoff_All(mm/day)_Delineated_Average'].sum() / runoff_obs['runoff_obs'].sum()
sim_mse = mean_squared_error(runoff_obs['runoff_obs'], velma_results['Runoff_All(mm/day)_Delineated_Average'])
sim_mae = mean_absolute_error(runoff_obs['runoff_obs'], velma_results['Runoff_All(mm/day)_Delineated_Average'])
display('MSE', sim_mse)
display('MAE', sim_mae)
display('RMSE', np.sqrt(sim_mse))

display('R2', r2_score(runoff_obs['runoff_obs'], velma_results['Runoff_All(mm/day)_Delineated_Average']))

'MSE'

40.02927863395068

'MAE'

2.4735576213197965

'RMSE'

6.326869576176727

'R2'

0.46738813770737575

## Plotting

In [5]:
# Group measurements by year
runoff_sim_yearly = pd.pivot_table(velma_results, index=['Day'], columns=['Year'],
                                   values=['Runoff_All(mm/day)_Delineated_Average'])
runoff_obs_yearly = pd.pivot_table(runoff_obs, index=['doy'], columns=['year'], values=['runoff_obs'])
precip_yearly = pd.pivot_table(precip, index=['doy'], columns=['year'], values=['precip'])

#### Observed vs. VELMA simulated runoff

VELMA simulated runoff consistently underpredicts the observed runoff during flow peaks. This could be sensor error or may be the flashiness of the stream. There is also a period of very low observed runoff during 2006 that looks fishy. 

In [15]:
# Observed vs. VELMA simulated runoff
years = runoff_obs_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for col, year in enumerate(years):
    runoff_obs_yearly.iloc[:, col].plot(ax=axes[col], label='Observed', linewidth=1)
    runoff_sim_yearly.iloc[:, col].plot(ax=axes[col], label='Simulated', linewidth=1)
    axes[col].set_title(year)
    axes[col].set_ylim([0, 80])
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3), fancybox=True, ncol=2)
axes[0].set_ylabel('Runoff (mm/day)')
fig.suptitle('Fig. 1: Observed vs. VELMA simulated runoff')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Observed vs. VELMA simulated runoff (with precipitation)

Including the precipitation suggests that the low flow values during 2006 are not from a drought but may instead be sensor error, because the precipitation during 2006 is comparable to the other years and shouldn't be causing such a low measurement period.

In [16]:
# Runoff and precip
years = runoff_obs_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))

for col, year in enumerate(years):
    ax2 = axes[col].twinx()
    precip_yearly.iloc[:, col].plot(ax=ax2, label='Precip', linewidth=0.5, color='tab:green')
    if col == 0:
        ax2.set_ylabel('Precipitation (mm/day)')
        ax2.legend(loc='upper right', bbox_to_anchor=(1, 1.3), fancybox=True, ncol=1)
    runoff_obs_yearly.iloc[:, col].plot(ax=axes[col], label='Observed', linewidth=1)
    runoff_sim_yearly.iloc[:, col].plot(ax=axes[col], label='Simulated', linewidth=1)
    ax2.invert_yaxis()
    axes[col].set_ylim([0, 80])
    axes[col].set_title(year)
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3))
axes[0].set_ylabel('Runoff (mm/day)')
fig.suptitle('Fig. 2: Observed vs. VELMA simulated runoff (with precipitation)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Observed vs. VELMA simulated runoff (with precipitation and quality flags for runoff measurements)

Here we can add in the quality flags for the runoff data. Each day has a flag, so we'll just include the ones that are the most unreliable. 

In [46]:
# Add quality scores below plot

colors = sns.color_palette('tab10', 10)

quality_edit = quality.copy()

quality_edit['doy'], quality_edit['year'] = quality_edit.index.dayofyear, quality_edit.index.year
quality_edit_yearly = pd.pivot_table(quality_edit, index=['doy'], columns=['year'], values=['Quality'])
quality2 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [-2, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
quality3 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [np.nan, -2, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
quality8 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                       [np.nan, np.nan, -2, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
quality10 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [np.nan, np.nan, np.nan, -2, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
quality50 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [np.nan, np.nan, np.nan, np.nan, -2, np.nan, np.nan, np.nan, np.nan, np.nan])
quality77 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [np.nan, np.nan, np.nan, np.nan, np.nan, -2, np.nan, np.nan, np.nan, np.nan])
quality160 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                         [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -2, np.nan, np.nan, np.nan])
quality161 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                         [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -2, np.nan, np.nan])
quality179 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                         [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -2, np.nan])
quality254 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                         [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -2])

years = runoff_obs_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))

for col, year in enumerate(years):
    ax2 = axes[col].twinx()
    precip_yearly.iloc[:, col].plot(ax=ax2, label='Precip', color='tab:green', linewidth=0.5)
    if col == 0:
        ax2.set_ylabel('Precipitation (mm/day)')
        ax2.legend(loc='upper right', bbox_to_anchor=(1, 1.3), fancybox=True, ncol=1)
    quality2.iloc[:, col].plot(ax=axes[col], color=colors[0], linewidth=2, label='Good provisional data')
    quality3.iloc[:, col].plot(ax=axes[col], color=colors[1], linewidth=2, label='Good provisional data - edited')
    quality8.iloc[:, col].plot(ax=axes[col], color=colors[2], linewidth=2, label='Below rating')
    quality10.iloc[:, col].plot(ax=axes[col], color=colors[3], linewidth=2, label='Above rating (<2x)')
    quality50.iloc[:, col].plot(ax=axes[col], color=colors[4], linewidth=2, label='Estimated')
    quality77.iloc[:, col].plot(ax=axes[col], color=colors[5], linewidth=2, label='Estimated from other station')
    quality160.iloc[:, col].plot(ax=axes[col], color=colors[6], linewidth=2, label='Above rating (>2x)')
    quality161.iloc[:, col].plot(ax=axes[col], color=colors[7], linewidth=2, label='Below rating (<1/2x)')
    quality179.iloc[:, col].plot(ax=axes[col], color=colors[8], linewidth=2, label='Estimated - unreliable')
    quality254.iloc[:, col].plot(ax=axes[col], color=colors[9], linewidth=2, label='Rating exceeded')
    runoff_obs_yearly.iloc[:, col].plot(ax=axes[col], label='Observed', linewidth=1)
    runoff_sim_yearly.iloc[:, col].plot(ax=axes[col], label='Simulated', linewidth=1)
    ax2.invert_yaxis()
    axes[col].set_ylim([-5, 80])
    axes[col].set_title(year)
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.9), fancybox=True, ncol=2)
axes[0].set_ylabel('Runoff (mm/day)')
fig.suptitle('Fig. 3: Observed vs. VELMA simulated runoff (with precipitation and quality flags)')
plt.tight_layout()
# plt.savefig('baseline_runoffs_qaflags.png', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Adding quality flags unreliable and extreme measurements, we can see that many of these are the peaks and troughs where VELMA performs the worst. This doesn't necessarily mean there is an error with the stream gauge, just flags the values as extra-ordinary. Does beg the question - is this the natural flashiness of Ellsworth that the rating table just didn't capture? Or is this the result of instrument error?

## Imputing runoff gauge measurements with low-quality tags

Imputing quality tags 160 (Above rating, over 2x), 161 (Below rating, less than 1/2x), and 254 (rating table exceeded).

The imputation process involves training an SVR model using precipitation (and 1-day lag and 2-day sums), temperature (mean, min, max), and sin and cosine day of year. The model is trained on the runoff dataset with low-quality values removed. Then the trained model is used to predict the removed values.

In [18]:
# Imputing select (but not all) days with quality codes

# Use model for runoff imputation to estimate removed values

low_qual = quality[(quality['Quality'].isin([2, 254, 179, 161]))]

flow = pd.read_csv(flow_path, usecols=['Date', 'Flow_cfs'], parse_dates=True, index_col=0)

# Convert streamflow from cfs to mm/day
# 2.446576 ft3/sec =  1m3/35.314667ft3 * 1/km2 * 86400sec/1day * 1km2/1000000m2 * 1000mm/1m
ft3_sec = (1 / 35.314667) * 86400 * (1 / 1000000) * 1000
area = 13.7393  # area of upstream Ellsworth watershed, sq. km
flow['flow_mm_day'] = (flow['Flow_cfs'] / area) * ft3_sec
flow.drop('Flow_cfs', axis=1, inplace=True)

# Expand date range to include every day of all the years present
begin = '01-01-{}'.format(flow.index.to_frame()['Date'].min().year)
end = '12-31-{}'.format(flow.index.to_frame()['Date'].max().year)
rng = pd.date_range(begin, end)
df = pd.DataFrame(index=rng)
daily_flow = df.merge(flow, left_index=True, right_index=True, how='left')

# Feature engineering
daily_precip_path = config.daily_ppt
precip = pd.read_csv(str(daily_precip_path), parse_dates=True, index_col=0)
daily_temp_mean_path = config.daily_temp_mean
temp_mean = pd.read_csv(str(daily_temp_mean_path), parse_dates=True, index_col=0)
daily_temp_min_path = config.daily_temp_min
temp_mean_min = pd.read_csv(str(daily_temp_min_path), parse_dates=True, index_col=0)
daily_temp_max_path = config.daily_temp_max
temp_mean_max = pd.read_csv(str(daily_temp_max_path), parse_dates=True, index_col=0)
df = pd.concat([precip, temp_mean, temp_mean_min, temp_mean_max], axis=1)

# Convert day of year to signal
day = 24 * 60 * 60
year = 365.2425 * day
timestamp_secs = pd.to_datetime(df.index)
timestamp_secs = timestamp_secs.map(datetime.datetime.timestamp)
df['year_cos'] = np.cos(timestamp_secs * (2 * np.pi / year))
df['year_sin'] = np.sin(timestamp_secs * (2 * np.pi / year))

# Sum of last 2 days precip
df['precip_sum-2t'] = precip.rolling(2).sum()

# Previous days' precip
df['precip_t-1'] = precip['mean_ppt_mm'].shift(1)
df['precip_t-2'] = precip['mean_ppt_mm'].shift(2)
df['precip_t-3'] = precip['mean_ppt_mm'].shift(3)

obs = df.merge(daily_flow['flow_mm_day'], left_index=True, right_index=True, how='right')

# Set aside dates with missing flow measurements
gap_data = obs[obs['flow_mm_day'].isna()]
obs.dropna(inplace=True)

# Remove days with low quality measurements
shared_index = obs.index.intersection(low_qual.index)
obs_highqual = obs.drop(shared_index)

def plot_test_results(y_test, y_pred):
    results = pd.DataFrame(data=np.column_stack([y_test, y_pred]), index=y_test.index, columns=['y_test', 'y_pred'])
    results = (results * train_std['flow_mm_day']) + train_mean['flow_mm_day']
    plt.plot(results)
    plt.legend(results.columns)


# Split the data 70-20-10
n = obs_highqual.shape[0]
train_df = obs_highqual[0:int(n * 0.7)]
test_df = obs_highqual[int(n * 0.7):]
num_features = obs_highqual.shape[1]

cols = obs_highqual.columns.tolist()
target = cols.index('flow_mm_day')

# Normalize
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

X_train, y_train = train_df.iloc[:, 0:target], train_df.iloc[:, target]
X_test, y_test = test_df.iloc[:, 0:target], test_df.iloc[:, target]

svr = SVR(kernel='rbf', C=1, gamma='auto', epsilon=0.1)
svr.fit(X_train, y_train)

y_pred = svr.predict(X_test)
display('MSE for SVR test data: ', mean_squared_error(y_test, y_pred))

plt.figure()
plot_test_results(y_test, y_pred)

'MSE for SVR test data: '

0.12769544285958093

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now we use the trained model to impute the low-quality values, combine the results with the reliable original data, and plot.

In [19]:
# Using trained model to imput the measurements removed for low quality

velma_start = pd.to_datetime('01-01-2004')
velma_end = pd.to_datetime('12-31-2007')

# Remove dates with poor quality tags/rating table exceedances
gap_data = low_qual.merge(df, left_index=True, right_index=True, how='left')
gap_data_04_07 = gap_data[(gap_data.index >= velma_start) & (gap_data.index <= velma_end)].copy()
X_gap = gap_data_04_07.drop(columns=['Quality'], axis=1)

X_gap = (X_gap - train_mean[:-1]) / train_std[:-1]
gap_pred = svr.predict(X_gap)
gap_pred = (gap_pred * train_std['flow_mm_day']) + train_mean['flow_mm_day']

gap_data_04_07['flow_mm_day'] = gap_pred

# Combine imputed flow with observed data
gap_data_04_07_imp = gap_data_04_07.drop(columns=['Quality', 'mean_ppt_mm', 'mean_temp_c', 'Mean', 'Mean', 'year_cos',
                                                  'year_sin', 'precip_sum-2t', 'precip_t-1', 'precip_t-2',
                                                  'precip_t-3'], axis=1).copy()
gap_data_04_07_imp['doy'], gap_data_04_07_imp['year'] = gap_data_04_07_imp.index.dayofyear, gap_data_04_07_imp.index.year

runoff_obs_drop = runoff_obs.drop(gap_data_04_07_imp.index)
runoff_obs_drop = runoff_obs_drop.rename(columns={'runoff_obs': 'flow_mm_day'})
runoff_obs_imp = pd.concat([runoff_obs_drop, gap_data_04_07_imp]).sort_index()

# Plot imputed data vs. VELMA simulated data
runoff_obs_imp_yearly = pd.pivot_table(runoff_obs_imp, index=['doy'], columns=['year'], values=['flow_mm_day'])

# Plot average daily runoff vs. VELMA simulated runoff
years = runoff_obs_imp_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for col, year in enumerate(years):
    runoff_obs_imp_yearly.iloc[:, col].plot(ax=axes[col], label='Observed (imputed)', linewidth=1)
    runoff_sim_yearly.iloc[:, col].plot(ax=axes[col], label='Simulated', linewidth=1)
    axes[col].set_title(year)
    axes[col].set_ylim([0, 80])
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3), fancybox=True, ncol=2)
axes[0].set_ylabel('Runoff (mm/day)')
fig.suptitle('Fig. 5: Observed vs. VELMA simulated runoff (imputed values)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# Computing Nash-Sutcliffe
def NS(s, o):
    """
        Nash Sutcliffe efficiency coefficient
        input:
        s: simulated
        o: observed
        output:
        ns: Nash Sutcliffe efficient coefficient
        """
    # s,o = filter_nan(s,o)
    return 1 - np.sum((s-o)**2)/np.sum((o-np.mean(o))**2)

display('NS (original runoff values): ', NS(velma_results['Runoff_All(mm/day)_Delineated_Average'], runoff_obs['runoff_obs']))

display('NS (low-quality values imputed): ', NS(velma_results['Runoff_All(mm/day)_Delineated_Average'], runoff_obs_imp['flow_mm_day']))

'NS (original runoff values): '

0.46738813770737575

'NS (low-quality values imputed): '

0.4783450553077594

### Removing unreliable runoff values and calculating Nash-Sutcliffe

Imputation improves the fit significantly, but it also introduces quite a bit of uncertainty.

What if we just removed the values that are the least reliable from computing Nash-Sutcliffe? The following quality flags were removed: "Exceeded rating table (>2x)" (160), "Below rating table (<1/2x)" (161), "Estimated - unreliable" (179).

In [49]:
runoff_sim = velma_results['Runoff_All(mm/day)_Delineated_Average']
df = pd.concat([runoff_sim, runoff_obs, quality], axis=1)
df_edited = df.drop(df[(df['Quality'] == 50) | (df['Quality'] == 160) | (df['Quality'] == 161) | (df['Quality'] == 179)].index)
df_edited = df.drop(df[(df['Quality'] == 160) | (df['Quality'] == 161) | (df['Quality'] == 179)].index)
df_edited = df.drop(df[(df['Quality'] == 160) | (df['Quality'] == 161) | (df['Quality'] == 179) | (df['Quality'] == 254)].index)

In [50]:
display('NSE (2004): ', NS(runoff_sim_yearly.iloc[:, 0], runoff_obs_yearly.iloc[:, 0]))
display('NSE (2005): ', NS(runoff_sim_yearly.iloc[:, 1], runoff_obs_yearly.iloc[:, 1]))
display('NSE (2006): ', NS(runoff_sim_yearly.iloc[:, 2], runoff_obs_yearly.iloc[:, 2]))
display('NSE (2007): ', NS(runoff_sim_yearly.iloc[:, 3], runoff_obs_yearly.iloc[:, 3]))
display('NSE (2004-2007): ', NS(velma_results['Runoff_All(mm/day)_Delineated_Average'], runoff_obs['runoff_obs']))

'NSE (2004): '

0.6005535538604774

'NSE (2005): '

0.7509217791510732

'NSE (2006): '

0.054723354364196175

'NSE (2007): '

0.579676568310854

'NSE (2004-2007): '

0.46738813770737575

NSE values after removing unreliable measurements

In [28]:
# 2003-2007 runoff
sim_yearly = pd.pivot_table(df_edited[['Runoff_All(mm/day)_Delineated_Average', 'doy', 'year']], 
                            index=['doy'], 
                            columns=['year'], 
                            values=['Runoff_All(mm/day)_Delineated_Average'])

obs_yearly = pd.pivot_table(df_edited[['runoff_obs', 'doy', 'year']], 
                            index=['doy'], 
                            columns=['year'], 
                            values=['runoff_obs'])

display('NS (2004): ', NS(sim_yearly.iloc[:, 0], obs_yearly.iloc[:, 0]))
display('NS (2005): ', NS(sim_yearly.iloc[:, 1], obs_yearly.iloc[:, 1]))
display('NS (2006): ', NS(sim_yearly.iloc[:, 2], obs_yearly.iloc[:, 2]))
display('NS (2007): ', NS(sim_yearly.iloc[:, 3], obs_yearly.iloc[:, 3]))

df_edited_04_07 = df_edited[(df_edited.index >= pd.to_datetime('2004-01-01'))]
display('NS (2004-2007): ', NS(df_edited_04_07['Runoff_All(mm/day)_Delineated_Average'], df_edited_04_07['runoff_obs']))
display('RMSE (2004-2007): ', np.sqrt(mean_squared_error(df_edited_04_07['Runoff_All(mm/day)_Delineated_Average'], df_edited_04_07['runoff_obs'])))


'NS (2004): '

0.6606872992039245

'NS (2005): '

0.8046441136760274

'NS (2006): '

...

0.579676568310854

'NS (2004-2007): '

0.5973812617184182

'RMSE (2004-2007): '

4.6259125648075

In [25]:
years = obs_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for col, year in enumerate(years):
    obs_yearly.iloc[:, col].plot(ax=axes[col], label='Observed (imputed)', linewidth=1)
    sim_yearly.iloc[:, col].plot(ax=axes[col], label='Simulated', linewidth=1)
    axes[col].set_title(year)
    axes[col].set_ylim([0, 80])
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3), fancybox=True, ncol=2)
fig.suptitle('Fig. 6: Observed vs. VELMA simulated runoff (unreliable values removed)')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Export runoff with quality flagged values as NaN

In [ ]:
# runoff_quality = runoff_obs.merge(quality, left_index=True, right_index=True, how='left')
# remove = runoff_quality.loc[(runoff_quality['Quality'] == 160) | (runoff_quality['Quality'] == 161) | (runoff_quality['Quality'] == 179)].index
# runoff_quality.loc[remove,'runoff_obs'] = 'NaN'

In [ ]:
# outfile = config.velma_data / 'runoff' / 'ellsworth_Q_2003_2007_dummy_flagged.csv'
# runoff_quality['runoff_obs'].to_csv(outfile, index=False, header=False)